In [280]:
%load_ext autoreload
%autoreload 2

from api.yelp.yelp import YelpAPI
from algo import *
from utils import *
from constraints import *
from plan import *
import datetime
import time

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [281]:
CAT_FILE = 'api/yelp/yelpcategories.json'
with open('api/yelp/yelpkey.txt', 'r') as f:
    API_KEY = f.read().strip()
api = YelpAPI(API_KEY, CAT_FILE)

In [282]:
mex = api.business_search(location="97 5th Avenue, New York, NY 10003", categories=['mexican'])

cafe = api.business_search(location="97 5th Avenue, New York, NY 10003", categories=['cafes'], add_parent_categories=True)

movies = api.business_search(location="97 5th Avenue, New York, NY 10003", categories=['movietheaters'], add_parent_categories=True)

In [300]:
movies['businesses'][1].categories

{'arts', 'movietheaters'}

In [284]:
for b in cafe['businesses']:
    print(b.name, b.distance, b.rating)

Eataly 488.0897963313967 4.0
Breads Bakery 143.50184336250797 4.5
Union Square Cafe 338.7121222183838 4.0
Daily Provisions 360.226374766106 4.5
La Lanterna Di Vittorio 1059.0680563302228 4.0
Brooklyn Bagel & Coffee Company 1041.7312658071687 4.0
Café Grumpy 755.5983617867082 4.0
The Bluebell Cafe 744.9626788474335 4.0
Bao Bao Cafe 778.8511501315928 4.0
Chobani SoHo 1560.9027955233576 4.5
Maison Kayser 306.0148063621218 3.5
Housing Works Bookstore Café 1529.2163434449587 4.5
Cafe Hanamizuki 1063.000303549232 4.0
Pasticceria Rocco 1197.2549664283115 4.0
Le Baratin 743.0116623539029 4.0
Patisserie Claude 1017.6984045553193 4.0
Birch Coffee 704.3867642065566 4.0
Outro 572.4780219339871 4.0
Ninth Street Espresso 1289.5219751871368 4.0
The Commons Chelsea 645.6914653519865 4.0


In [285]:
final_businesses = dedupe_list(mex['businesses'] + cafe['businesses'] + movies['businesses'], lambda x: x.id)
details = []
for biz in final_businesses:
    details.append(api.business_details(biz.id))
    time.sleep(0.1)

In [316]:
entities = []
for detail in details:
    entities.append(Entity(business=detail))

In [322]:
G = build_business_graph(entities, [["cafes", "mexican", "movietheaters"], ["cafes", "mexican", "movietheaters"]], min_rating=3.0, max_distance=2000, min_dollar_signs=None, max_dollar_signs=None)

In [318]:
len(cafe['businesses']), len(mex['businesses']), len(final_businesses)

(20, 20, 60)

In [319]:
def print_edge(G, nodeA, nodeB):
    edge = G.get_edge_properties(nodeA, nodeB)
    print("{0} ({1}) --> {2} ({3})  ({4})".format(nodeA.entity.name, nodeA.entity.price, nodeB.entity.name, nodeB.entity.price, edge['dist']))

In [315]:
for n in G.nodes:
    edges = G.get_edges(n)
    for n2 in edges:
        pass #print_edge(G, n, n2)

In [88]:
details = api.business_details(cafe['businesses'][0].id)

In [92]:
details.hours

[[{'end': datetime.time(23, 0),
   'is_overnight': False,
   'start': datetime.time(9, 0)}],
 [{'end': datetime.time(23, 0),
   'is_overnight': False,
   'start': datetime.time(9, 0)}],
 [{'end': datetime.time(23, 0),
   'is_overnight': False,
   'start': datetime.time(9, 0)}],
 [{'end': datetime.time(23, 0),
   'is_overnight': False,
   'start': datetime.time(9, 0)}],
 [{'end': datetime.time(23, 0),
   'is_overnight': False,
   'start': datetime.time(9, 0)}],
 [{'end': datetime.time(23, 0),
   'is_overnight': False,
   'start': datetime.time(9, 0)}],
 [{'end': datetime.time(23, 0),
   'is_overnight': False,
   'start': datetime.time(9, 0)}]]

In [93]:
x = time.strptime('1225', "%H%M")
datetime.time(x.tm_hour, x.tm_min)

datetime.datetime(2006, 11, 21, 16, 30).time() > datetime.time(x.tm_hour, x.tm_min)

True

In [94]:
y = 4
3 < y < 7

True

In [95]:
z = datetime.timedelta(seconds=1800)

In [96]:
datetime.datetime(2006, 11, 21, 16, 30) + z

datetime.datetime(2006, 11, 21, 17, 0)

In [332]:
u = UberConstraints()
b = BooleanConstraints()
se = StateExtractors()
sc = StateConstraints()
ee = EdgeExtractors()
seq_constraint = b.bool_and(
    [
        b.bool_less_than(se.get_attr("num_prev_states"), se.const(3)),
        u.follows_cat_sequence([set([x]) for x in ["mexican", "movietheaters", "mexican"]]),
        b.bool_less_than(ee.get_distance(use_miles=True), se.const(0.3)),
        sc.no_repeat_visits()
    ]
)

def time_spent_fn(entity):
    if "movietheaters" in entity.entity.categories:
        return [datetime.timedelta(hours=3)]
    return [datetime.timedelta(hours=1.5)]

done_constraint = b.bool_equals(se.get_attr("num_prev_states"), se.const(2))
init_constraint = sc.curr_state_category_in(set(["mexican"]))

date_start = datetime.datetime(2018, 12, 6, 18, 30)
initial_states = build_initial_states(G, [date_start], init_constraint)

neighbor_fn = build_neighbor_state_fn(seq_constraint, time_spent_fn=time_spent_fn)

success_fn = build_success_state_fn(done_constraint)

In [333]:
results = list(generate_plans(G, initial_states, neighbor_fn, success_fn, None))

In [334]:
def print_results(results):
    # t_format = "%m/%d/%y %B %Y %H:%M"
    t_format = "%I:%M %p"
    for state in results:
        all_prev = state.prev_state_list()
        out = []
        for s in (all_prev + [state]):
            out.append("{} ({} - {})".format(s.node.entity.name, s.start_dt.strftime(t_format), s.end_dt.strftime(t_format)))
        print(" => ".join(out))
        print()
print_results(results)

DORADO Tacos & Quesadillas (06:30 PM - 08:00 PM) => Quad Cinema (08:03 PM - 11:03 PM) => Tortaria (11:07 PM - 12:37 AM)



In [307]:
state = results[0]
state.prev_state_list()
all_states = state.prev_state_list() + [state]
print(G.get_edge_properties(all_states[1].node, all_states[2].node))

IndexError: list index out of range

In [246]:
initial_names = set([state.node.entity.name for state in initial])
cafe_names = set([node.name for node in cafe['businesses']])

In [142]:
print(len(initial_names), len(cafe_names))
print()
print(initial_names)
print()
print(cafe_names.difference(initial_names))

2 20

{'La Lanterna Di Vittorio', 'Pasticceria Rocco'}

{'Daily Provisions', 'Breads Bakery', 'Eataly', 'Ninth Street Espresso', 'Birch Coffee', 'Le Baratin', 'Maison Kayser', 'Outro', 'Chobani SoHo', 'The Bluebell Cafe', 'Bao Bao Cafe', 'Union Square Cafe', 'Housing Works Bookstore Café', 'The Commons Chelsea', 'Cafe Hanamizuki', 'Patisserie Claude', 'Café Grumpy', 'Brooklyn Bagel & Coffee Company'}


In [127]:
for c in cafe['businesses']:
    print(c.name, c.id)

Eataly a0IET3_yCFcO36OqGSsisg
Breads Bakery mU23NlQfn1R9OhA6WMe_xg
Union Square Cafe lXGCo-K7eftxRT_6E59-Iw
Daily Provisions t4M1VX4n34eEgUWMJ7cgKw
La Lanterna Di Vittorio V-FCEicobl5Su-R7EVJQNA
Brooklyn Bagel & Coffee Company pb_ryc8FYHpUPV56eV2Eyg
Café Grumpy IYDE89R_ETmVP1-e82Y_xQ
The Bluebell Cafe 63nYipWuA42RZmESBMUNkQ
Bao Bao Cafe ynIx549QFj29o84y1RSGyw
Chobani SoHo z9sRnN1gPyxz9SJ52XDWoQ
Maison Kayser bhO25n-KN55UrqSOAb9CUQ
Housing Works Bookstore Café O4UeWRZLKuS3GRC4ELJRog
Cafe Hanamizuki S-619sO8qSxQcis0DBHvNw
Pasticceria Rocco JWLRT0dkdaC7AHtYapWbGw
Le Baratin WoZnkXZInggzgQM_LbDpGg
Patisserie Claude fvDZVAvv_g4L-WWkbDpCXg
Birch Coffee cvETUmgjYfjcAgl54aWntA
Outro IiLLntICUim7YREkIBU7lA
Ninth Street Espresso JqIrbpx-gysImU1vUgULPw
The Commons Chelsea 9BM1-Obd5Dr8pErgyBbQAQ


<function algo.build_neighbor_state_fn.<locals>.neighbor_state_fn>

[]